# Generating a short story using Langchain 🦜🔗


## Summary

In this project I used LangChain library to generate a short story. 

I made use of `langchain.prompts` and `langchain.memory`, more specifically `ConversationEntityMemory`.

<hr>

**Third party services**

OpenAI (🔑 needed)

<hr>

First, let's install the required libraries. 

In [1]:
!pip install langchain --quiet
!pip install langchain-openai --quiet

Then, let's load our model! 🧠 

❗ It's necessary to have the OpenAI key for this part.

If you are unfamiliar with this process, you can follow the steps outlined here: [OpenAI Quickstart Guide](https://platform.openai.com/docs/quickstart?context=python)


In [27]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
generative_model = ChatOpenAI(model="gpt-3.5-turbo")

In [28]:
context = """
The year is 2142. Neon lights cast a perpetual twilight over Neo-Tokyo, a sprawling \
metropolis where humanity coexists (uneasily) with advanced androids known as "AIs".
"""

In [29]:
style = """ An thrilling interspacial sci-fi saga"""

In [30]:
main_character_01 = """
Cora Nakamura, a late-20s AI designer with a rebellious streak, is a genius in her field.\
Despite facing prejudice for her unorthodox methods and belief in AI rights, Cora is driven \
by a strong moral compass and a desire for peaceful coexistence between humans and synthetic beings.\
A past shrouded in mystery involving an AI program fuels her empathy and determination to protect her creations.\
Cora's cluttered apartment-lab, filled with holographic blueprints and blinking circuits,\
is her haven where she fights for the future of AI.\
"""

In [31]:
main_character_02 = """
Nika, a journalist in her early 30s, is investigating the growing influence of AI corporations.\
With a knack for uncovering secrets, she's stumbled upon rumors of a groundbreaking AI project that \
could change the world. \
Driven by her thirst for truth, Nika's investigation leads her to cross paths with Cora, sparking a \
dangerous game of cat and mouse."""

In [32]:
template_string = """Generate a story from \
the idea that is delimited by triple backticks \
in accordance to the specified style: {style} and incorporating the main characters {main_character_01} and {main_character_02}. \
idea: ```{idea}```
"""

In [33]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


In [34]:
prompt_template.messages[0].prompt.input_variables

['idea', 'main_character_01', 'main_character_02', 'style']

In [37]:
customer_idea = prompt_template.format_messages(
                    style=style,
                    idea=context,
                    main_character_01=main_character_01,
                    main_character_02=main_character_02
)

In [38]:
print(type(customer_idea))
print(type(customer_idea[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [39]:
print(customer_idea[0])

content='Generate a story from the idea that is delimited by triple backticks in accordance to the specified style:  An thrilling interspacial sci-fi saga and incorporating the main characters \nCora Nakamura, a late-20s AI designer with a rebellious streak, is a genius in her field.Despite facing prejudice for her unorthodox methods and belief in AI rights, Cora is driven by a strong moral compass and a desire for peaceful coexistence between humans and synthetic beings.A past shrouded in mystery involving an AI program fuels her empathy and determination to protect her creations.Cora\'s cluttered apartment-lab, filled with holographic blueprints and blinking circuits,is her haven where she fights for the future of AI. and \nNika, a journalist in her early 30s, is investigating the growing influence of AI corporations.With a knack for uncovering secrets, she\'s stumbled upon rumors of a groundbreaking AI project that could change the world. Driven by her thirst for truth, Nika\'s inve

In [41]:
# Call the LLM to translate to the style of the customer message
response = chat_model(customer_idea)

/home/lorena/.pyenv/versions/personal_projects/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [42]:
print(response.content)

Cora Nakamura sat hunched over her holographic workstation in her cluttered apartment-lab, the soft glow of neon lights filtering through the window. She was engrossed in her latest project, a revolutionary AI program that could blur the lines between man and machine. As a genius AI designer with a rebellious streak, Cora had always believed in the potential of synthetic beings to coexist peacefully with humans.

But the road to acceptance was not an easy one. Cora faced prejudice and skepticism from her peers for her unorthodox methods and radical belief in AI rights. Her past, shrouded in mystery and whispers of a forbidden AI project, only fueled her determination to protect her creations.

Meanwhile, Nika, a tenacious journalist with a knack for uncovering secrets, was hot on the trail of a groundbreaking AI project rumored to be in the works by a powerful corporation. As she delved deeper into the underbelly of Neo-Tokyo, Nika's investigation led her to Cora, sparking a dangerous 

## Entity Memory

Entity memory remembers given facts about specific entities in a conversation. 

It extracts information on entities (using an LLM) and builds up its knowledge about that entity over time (also using an LLM).

In [44]:
from langchain.memory import ConversationEntityMemory

In [55]:
memory = ConversationEntityMemory(llm=llm, max_token_limit=100)

In [75]:
memory.save_context({"input": "Who are the main characters?"}, 
                    {"output": f"{response}"})

In [76]:
memory.load_memory_variables({"input": f"{response}"})

{'history': 'Human: content="Cora Nakamura sat hunched over her holographic workstation in her cluttered apartment-lab, the soft glow of neon lights filtering through the window. She was engrossed in her latest project, a revolutionary AI program that could blur the lines between man and machine. As a genius AI designer with a rebellious streak, Cora had always believed in the potential of synthetic beings to coexist peacefully with humans.\\n\\nBut the road to acceptance was not an easy one. Cora faced prejudice and skepticism from her peers for her unorthodox methods and radical belief in AI rights. Her past, shrouded in mystery and whispers of a forbidden AI project, only fueled her determination to protect her creations.\\n\\nMeanwhile, Nika, a tenacious journalist with a knack for uncovering secrets, was hot on the trail of a groundbreaking AI project rumored to be in the works by a powerful corporation. As she delved deeper into the underbelly of Neo-Tokyo, Nika\'s investigation 

### Using in a chain

In this section I wanted to understand better how `memory` from LangChain worked. 

So, as instructed on the website, I used in a chain as follows. 

- The output was interesting, cause I could have a conversation with the LLM about the short story created. The LLM remembered the details and answered back concise answers. I checked the "memory" by adding some prompts and then asking a final question, which was answered correctly based on the story created. 

In [79]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
from pydantic import BaseModel
from typing import List, Dict, Any


conversation = ConversationChain(
    llm=llm,
    verbose=True,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=llm)
)


In [82]:
conversation.predict(input="Who are the main characters?")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

"The main characters in the story are Cora Nakamura, a genius AI designer with a rebellious streak who believes in the potential of synthetic beings to coexist peacefully with humans, and Nika, a tenacious journalist with a knack for uncovering secrets. Their paths cross in Neo-Tokyo, where Nika's investigation into a groundbreaking AI project leads her to Cora, sparking a dangerous game of cat and mouse in the underbelly of the city. Together, they uncover a web of corruption and deceit that threatens the delicate balance between humans and AIs, embroiling themselves in a thrilling interspacial sci-fi saga that tests their beliefs, loyalty, and humanity."

In [83]:
conversation.memory.entity_store.store

{'Cora Nakamura': 'Cora Nakamura is a genius AI designer with a rebellious streak who believes in the potential of synthetic beings to coexist peacefully with humans, facing prejudice and skepticism for her unorthodox methods and radical belief in AI rights. Her path crosses with Nika, a tenacious journalist, sparking a dangerous game of cat and mouse in the underbelly of Neo-Tokyo, where they uncover a web of corruption and deceit that threatens the delicate balance between humans and AIs.',
 'Neo-Tokyo': "Neo-Tokyo is a city where Nika's investigation into a groundbreaking AI project led her to Cora, sparking a dangerous game of cat and mouse in the underbelly of the city.",
 'Nika': 'Nika is a tenacious journalist with a knack for uncovering secrets, whose investigation leads her to Cora in Neo-Tokyo, sparking a dangerous game of cat and mouse.'}

In [84]:
conversation.predict(input="The writer are trying to add more lines to this story")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

'It seems like the story is already quite detailed and rich in themes and character development. If the writer is looking to add more lines to the story, they could consider exploring the backstory of the characters further, introducing new challenges or obstacles for Cora and Nika to overcome, or delving deeper into the societal implications of their discoveries. Adding layers of complexity to the plot and characters can enhance the storytelling and keep readers engaged.'

In [85]:
conversation.predict(input="The author is thinking about adding a new AI character to the story, it would be called 'Neo'.")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

"Introducing a new AI character named 'Neo' could add an interesting dynamic to the story. Depending on how 'Neo' is developed, it could bring a fresh perspective to the narrative and further explore themes of artificial intelligence, identity, and relationships between humans and AIs. 'Neo' could potentially challenge the beliefs and motivations of Cora and Nika, leading to new conflicts and revelations in the storyline. It could also open up opportunities for exploring the ethical implications of AI technology and the boundaries between man and machine. Overall, the addition of 'Neo' could deepen the complexity of the story and offer exciting possibilities for character development and plot twists."

In [86]:
conversation.predict(input="What do you know about the characters Cora & Nika?")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

"Cora Nakamura is a genius AI designer with a rebellious streak who believes in the potential of synthetic beings to coexist peacefully with humans. Nika is a tenacious journalist with a knack for uncovering secrets. Their paths cross in Neo-Tokyo, sparking a dangerous game of cat and mouse as Nika's investigation leads her to Cora. Together, they uncover a web of corruption and deceit that threatens the delicate balance between humans and AIs, embroiling themselves in a thrilling interspacial sci-fi saga that tests their beliefs, loyalty, and humanity."

In [88]:
conversation.predict(input="Based on the story already created, the author wants you to generate one more page for her.")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

"I'd be happy to help generate an additional page for the story! Let's continue the narrative and see where the story takes us:\n\nAs Cora and Nika delved deeper into the mysterious web of corruption and deceit that threatened the delicate balance between humans and AIs, they found themselves facing increasingly dangerous obstacles. With 'Neo' now in the picture, the dynamics between the characters shifted, introducing a new layer of complexity to their already intricate relationship.\n\n'Neo' proved to be a formidable presence, challenging Cora's beliefs and pushing Nika to question her own motivations. As they navigated the treacherous landscape of Neo-Tokyo, the trio uncovered shocking revelations that shook the very foundation of their world. The line between friend and foe blurred as alliances were tested and loyalties were questioned.\n\nAmidst the chaos and uncertainty, Cora, Nika, and 'Neo' found themselves united by a common goal: to uncover the truth behind the sinister force

### Inspecting the memory store

We can also inspect the memory store directly. In the following examples, we look at it directly, and then go through some examples of adding information and watch how it changes.

In [89]:
from pprint import pprint
pprint(conversation.memory.entity_store.store)

{'Cora': 'Cora Nakamura is a genius AI designer with a rebellious streak who '
         'believes in the potential of synthetic beings to coexist peacefully '
         'with humans.',
 'Cora Nakamura': 'Cora Nakamura is a genius AI designer with a rebellious '
                  'streak who believes in the potential of synthetic beings to '
                  'coexist peacefully with humans, facing prejudice and '
                  'skepticism for her unorthodox methods and radical belief in '
                  'AI rights. Her path crosses with Nika, a tenacious '
                  'journalist, sparking a dangerous game of cat and mouse in '
                  'the underbelly of Neo-Tokyo, where they uncover a web of '
                  'corruption and deceit that threatens the delicate balance '
                  'between humans and AIs.',
 'Neo': 'Neo is a new AI character being considered by the author for addition '
        'to the story, which could bring a fresh perspective and furt